# Tutorial for Toy Generation  to be used for limit production

#### Some intro here...  //FIXME TODO

First of all I suggest you to produce a single compilable macro that contains the likelihood you want to use, you can find in other tutorial how to define the likelihood [here](https://xenon1t.github.io/Xephyr/docs/tutorials.html). The idea of this macro is to define a function that you can use to retrieve the likelihood, so that you have it defined in one place only. You can get inspiration on how to write this function from [here](https://github.com/XENON1T/SR1Results/blob/master/StatisticalAnalyses/xephyr_sr1_likelihood/src/likelihoodDef.cxx)

In [1]:
.x loadXephyr.C

(int) 0


In [2]:
// add path to include files to ROOT (in the file I'm going to load there are some Xephyr includes) 
gInterpreter->AddIncludePath("../src");  // in this case is just XEPHYR src from next dir.

// now load the function to retrieve the likelihood

In [3]:
.L ../../SR1/StatisticalAnalyses/xephyr_sr1_likelihood/src/likelihoodDef.cxx

In [4]:
pdfLikelihood likeHood = getTheLikelihood(50.);  // 50. GeV mass

pdfLikelihood - INFO: bkg component named hband added to    SAFEGUARDED
pdfLikelihood::initialize - INFO :  initialize..... 
	Likelihood::addParameter - Info : Adding parameter Sigma  with ID -1  to PL xe1T
INFO :  adding sys for BKG component hband
	Likelihood::addParameter - Info : Adding parameter ERscale  with ID 1  to PL xe1T
	Likelihood::addParameter - Info : Adding parameter _PY  with ID 2  to PL xe1T
	Likelihood::addParameter - Info : Adding parameter _RF  with ID 3  to PL xe1T
INFO :  adding sys for SIGNAL component wimp_0050gev
	Likelihood::addParameter - Info : Adding parameter _gamma_  with ID 4  to PL xe1T
	Likelihood::addParameter - Info : Adding parameter _alpha_  with ID 5  to PL xe1T
	Likelihood::addParameter - Info : Adding parameter _eta_  with ID 6  to PL xe1T
	Likelihood::addParameter - Info : Adding parameter _acceptance_par_  with ID 7  to PL xe1T

------ WARNING -------  Safeguard is turned OFF altough you have set components to be safeguarded this is ignored --

In [5]:
TString treeName = "treeName";
TString pathOfFile = "./";
ToyGenerator toy(treeName,pathOfFile);  // (string 'name', string 'out dir')

In [6]:
toy.setLikelihood(&likeHood);

In [7]:
toy.setSeed(127);    // you should change the seed as a function of wimp mass and injected signal strenght

In [8]:
toy.randomizeNuissanceParameter();

ToyGenerator::randomizeNuissanceParameter - INFO : Randomizing parameters:
ToyGenerator:: - INFO : Skipping paramater: Sigma
ToyGenerator:: - WARNING : Following parameter is FREE and will be extracted from uniform distro. ERscale
ToyGenerator:: - INFO : Skipping paramater: _gamma_
ToyGenerator:: - INFO : Skipping paramater: _eta_
 Id Name                 Type         Current Value               Uncertainty
 -1 Sigma                Of interest        0                 
  1 ERscale              Free            0.24                 
  2 _PY                  Nuisance           1                 
  3 _RF                  Nuisance        -1.3                 
  4 _gamma_              Fixed              0                 
  5 _alpha_              Nuisance       -0.68                 
  6 _eta_                Fixed              0                 
  7 _acceptance_par_     Nuisance        0.38                 


In [9]:
toy.generateData(0.1,10);

### Let's have a look at what we've created
In the specified directory you'll get a file named with your specified "TreeName" with N trees inside.
Each tree contains a list of the parameter values that have been extracted and considered as the truth for that toy dataset.

In [10]:
TFile *fi = TFile::Open(pathOfFile + treeName + ".root");
fi->ls();

TFile**		./treeName.root	
 TFile*		./treeName.root	
  KEY: TTree	treeName;1	generated toy data


In [11]:
TTree *tree = (TTree*)fi->Get(treeName);

In [12]:
TList *l = tree->GetUserInfo();  // this is a list stored in TTree
l->Print();                      // let's print the params for one tree, they are stored in TParameter class type.

Collection name='UserInfo', class='TList', size=8
  TParameter<double>	Sigma = 0.1
  TParameter<double>	ERscale = 0.238333
  TParameter<double>	_PY = 1.01894
  TParameter<double>	_RF = -1.25982
  TParameter<double>	_gamma_ = 0
  TParameter<double>	_alpha_ = -0.675689
  TParameter<double>	_eta_ = 0
  TParameter<double>	_acceptance_par_ = 0.380706
